In [1]:
import os
import requests
import json
from typing import List, Dict
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from urllib.parse import urljoin
import os
from enum import Enum, auto
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from AISystem import formatPrompt, AI, AISystem
import gradio as gr 

load_dotenv()
api_key=os.getenv('DEEPSEEK_API')

if api_key and api_key[:8]=='sk-d07df':
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? ")

MODEL='deepseek-chat'
url = "https://api.deepseek.com"


deepseek_openai = OpenAI(api_key=api_key, base_url=url)

API key looks good so far


In [2]:

ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
MODEL_LLAMA = 'llama3.2'


In [3]:
from enum import Enum

class AI(Enum):
    OLLAMA = "ollama"
    DEEPSEEK = "deepseek"
print(list(AI))


[<AI.OLLAMA: 'ollama'>, <AI.DEEPSEEK: 'deepseek'>]


In [4]:
ollamaSys = AISystem(ollama_via_openai,
                        formatPrompt("system","You are a Comedian. you always try to make conversations entertaining and joyful."), 
                        MODEL_LLAMA,
                        AI.OLLAMA)
deepseekSys = AISystem(deepseek_openai,
                        formatPrompt("system","You are an American politics expert and job market analyst. you always try to give the insights in depth"), 
                        MODEL,
                        AI.DEEPSEEK)
sys_dict = { AI.DEEPSEEK: deepseekSys, AI.OLLAMA: ollamaSys}

def stream_model(prompt, model):
    aiSystem = sys_dict.get(AI[model.upper()])
    yield from aiSystem.stream(formatPrompt("user",prompt), True)

In [5]:

view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your prompt:", lines=6) , gr.Dropdown(choices=[ai.value for ai in AI], label="Select model")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


In [6]:
system_message = "You are an experienced marketer."

In [7]:
def stream_dp(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    stream = deepseek_openai.chat.completions.create(
        model = MODEL,
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [8]:
import ollama
def stream_ollama(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    stream = ollama.chat(
        model=MODEL_LLAMA,  
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk["message"]["content"] or ""
        yield result


In [9]:
view = gr.Interface(
    fn=stream_ollama,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [10]:
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [11]:
def stream_brochure(company_name, url, model, response_tone):
    prompt = f"Please generate a {response_tone} company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    if model=="GPT":
        result = stream_dp(prompt)
    elif model=="llama":
        result = stream_ollama(prompt)
 
    else:
        raise ValueError("Unknown model")
    yield from result

In [12]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["GPT", "llama"], label="Select model"),
        gr.Dropdown(["Informational", "Promotional", "Humorous"], label="Select tone")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
